In [28]:
import pandas as pd
import openpyxl
df_in = pd.read_excel(r'C:\Users\user\Desktop\project\Inbound & outbound stats in Taiwan\表1-2-歷年來臺旅客按居住地分.xlsx', engine='openpyxl')
df_in.head()


,年\n度,居住地\nResidence,香港.澳門 HongKong. Macao,大陸 Mainland China,日本 Japan,韓國 Korea,印度 India,中東 Middle East,馬來西亞 Malaysia,新加坡 Singapore,...,歐洲合計 Total,澳大利亞 Australia,紐西蘭 New Zealand,大洋洲其他地區 Others,大洋洲合計 Total,南非 S. Africa,非洲其他地區 Others,非洲合計 Total,未列明 Unstated,總計 Grand Total
0,地區,NaN,亞洲地區,亞洲地區,亞洲地區,亞洲地區,亞洲地區,亞洲地區,亞洲地區,亞洲地區,...,歐洲地區,大洋洲,大洋洲,大洋洲,大洋洲,非洲地區,非洲地區,非洲地區,未列明 Unstated,NaN
1,69年1980,NaN,224663,0,667078,29615,3752,0,68794,53641,...,58622,13082,1493,287,14862,5295,4287,9582,4,1393254.0
2,70年1981,NaN,263257,0,603117,31088,4087,0,70105,59873,...,63214,14669,1895,379,16943,13858,4869,18727,4210,1409465.0
3,71年1982,NaN,298037,0,579507,41029,4851,0,63424,58044,...,62667,16954,2030,408,19392,13905,4416,18321,7373,1419178.0
4,72年1983,NaN,283466,0,598574,39622,4649,11538,70493,62004,...,65809,17165,2063,349,19577,13212,5076,18288,6657,1457404.0


In [47]:
value_vars_list = list(df_in)
del value_vars_list[0:2]
df_in_melt = df_in.melt(id_vars=['年\n度'], value_vars=value_vars_list, var_name='Residence', value_name='Count')
df_in_melt

,年\n度,Residence,Count
0,地區,香港.澳門 HongKong. Macao,亞洲地區
1,69年1980,香港.澳門 HongKong. Macao,224663
2,70年1981,香港.澳門 HongKong. Macao,263257
3,71年1982,香港.澳門 HongKong. Macao,298037
4,72年1983,香港.澳門 HongKong. Macao,283466
...,...,...,...
1973,106年2017,總計 Grand Total,1.07396e+07
1974,107年2018,總計 Grand Total,1.10667e+07
1975,108年2019,總計 Grand Total,1.18641e+07
1976,109年2020,總計 Grand Total,1.37786e+06


In [48]:
df_in_melt.rename({'年\n度':'Year'}, axis=1, inplace=True)
df_in_region = df_in_melt[df_in_melt['Year']=='地區']
region_dict={}
for row in df_in_region.iterrows():
    region_dict[row[1]['Residence']] = row[1]['Count']
region_dict
df_in_melt['Region'] = df_in_melt['Residence'].apply(lambda x: region_dict[x])
df_in_melt

,Year,Residence,Count,Region
0,地區,香港.澳門 HongKong. Macao,亞洲地區,亞洲地區
1,69年1980,香港.澳門 HongKong. Macao,224663,亞洲地區
2,70年1981,香港.澳門 HongKong. Macao,263257,亞洲地區
3,71年1982,香港.澳門 HongKong. Macao,298037,亞洲地區
4,72年1983,香港.澳門 HongKong. Macao,283466,亞洲地區
...,...,...,...,...
1973,106年2017,總計 Grand Total,1.07396e+07,NaN
1974,107年2018,總計 Grand Total,1.10667e+07,NaN
1975,108年2019,總計 Grand Total,1.18641e+07,NaN
1976,109年2020,總計 Grand Total,1.37786e+06,NaN


In [49]:
df_in_melt.dropna(subset=['Year'], inplace=True)
df_in_melt.drop(df_in_melt[df_in_melt['Year']=='地區'].index, inplace=True)
df_in_melt['Year']=df_in_melt['Year'].apply(lambda x: x[-4:])
df_in_melt.drop(df_in_melt[df_in_melt['Residence'].str.contains('計|區|明')].index, inplace=True)
df_in_melt['Residence_Ch'] = df_in_melt['Residence'].apply(lambda x: x.split(' ')[0])
df_in_melt['Residence'] = df_in_melt['Residence'].apply(lambda x: x.replace(x.split(' ')[0], ''))
df_in_melt.loc[df_in_melt[df_in_melt['Residence']==' HongKong. Macao'].index,'Residence'] = 'Hong Kong'
df_in_melt.loc[df_in_melt[df_in_melt['Residence_Ch']=='香港.澳門'].index,'Residence_Ch'] = '香港'
df_in_melt

,Year,Residence,Count,Region,Residence_Ch
1,1980,Hong Kong,224663,亞洲地區,香港
2,1981,Hong Kong,263257,亞洲地區,香港
3,1982,Hong Kong,298037,亞洲地區,香港
4,1983,Hong Kong,283466,亞洲地區,香港
5,1984,Hong Kong,279377,亞洲地區,香港
...,...,...,...,...,...
1800,2016,S. Africa,5077,非洲地區,南非
1801,2017,S. Africa,5757,非洲地區,南非
1802,2018,S. Africa,5596,非洲地區,南非
1803,2019,S. Africa,5872,非洲地區,南非


In [50]:
for col in list(df_in_melt):
    print(df_in_melt[col].value_counts())


2009    32
1988    32
1996    32
1984    32
1986    32
1991    32
1998    32
2003    32
1989    32
2020    32
2004    32
1993    32
2017    32
2008    32
1995    32
1980    32
2015    32
2010    32
2007    32
1985    32
1997    32
2011    32
2019    32
2006    32
1990    32
1987    32
2001    32
1983    32
2018    32
1992    32
1994    32
2013    32
2014    32
2016    32
1982    32
1999    32
2000    32
2005    32
1981    32
2002    32
2012    32
Name: Year, dtype: int64
 Canada            41
 Indonesia         41
 Vietnam           41
 Italy             41
 Singapore         41
 U.K.              41
 Philippines       41
 Mexico            41
 Middle East       41
Hong Kong          41
 Brazil            41
 Mainland China    41
 Korea             41
 Greece            41
 Malaysia          41
 Australia         41
 Sweden            41
 Belgium           41
 Russian           41
 France            41
 S. Africa         41
 Argentina         41
 Spain             41
 New Zealand      

In [61]:
df_out = pd.read_excel(r'C:\Users\user\Desktop\project\Inbound & outbound stats in Taiwan\表2-2-歷年中華民國國民出國目的地人次統計.xlsx', engine='openpyxl')
df_out.head()

value_vars_list_out = list(df_out)
del value_vars_list_out[0]
df_out_melt = df_out.melt(id_vars=['Year'], value_vars=value_vars_list_out, var_name='Destination', value_name='Count')
df_out_melt

df_out_region = df_out_melt[df_out_melt['Year']=='Region']
region_dict_out={}
for row in df_out_region.iterrows():
    region_dict_out[row[1]['Destination']] = row[1]['Count']

df_out_melt['Region'] = df_out_melt['Destination'].apply(lambda x: region_dict_out[x])

df_out_melt.drop(df_in_melt[df_in_melt['Year']=='Region'].index, inplace=True)
df_out_melt['Year']=df_in_melt['Year'].apply(lambda x: x[-4:])
df_out_melt.dropna(subset=['Year'], inplace=True)
df_out_melt

,Year,Destination,Count,Region
1,1980,香港\nHong Kong,63812,亞洲地區 ASIA
2,1981,香港\nHong Kong,72418,亞洲地區 ASIA
3,1982,香港\nHong Kong,81558,亞洲地區 ASIA
4,1983,香港\nHong Kong,98754,亞洲地區 ASIA
5,1984,香港\nHong Kong,109075,亞洲地區 ASIA
...,...,...,...,...
1671,2016,非洲合計 Total,5206,非洲 Africa
1672,2017,非洲合計 Total,16740,非洲 Africa
1673,2018,非洲合計 Total,16649,非洲 Africa
1674,2019,非洲合計 Total,27,非洲 Africa


In [62]:
df_out_melt.drop(df_out_melt[df_out_melt['Destination'].str.contains('計|區')].index, inplace=True)
df_out_melt['Destination_ch'] = df_out_melt['Destination'].apply(lambda x: x.split('\n')[0] if '\n' in x else x.split(' ')[0])
df_out_melt['Destination'] = df_out_melt['Destination'].apply(lambda x: x.replace(x.split('\n')[0], '').replace('\n', '') if '\n' in x else x.replace(x.split(' ')[0], ''))
df_out_melt

,Year,Destination,Count,Region,Destination_ch
1,1980,Hong Kong,63812,亞洲地區 ASIA,香港
2,1981,Hong Kong,72418,亞洲地區 ASIA,香港
3,1982,Hong Kong,81558,亞洲地區 ASIA,香港
4,1983,Hong Kong,98754,亞洲地區 ASIA,香港
5,1984,Hong Kong,109075,亞洲地區 ASIA,香港
...,...,...,...,...,...
1456,2016,Palau,14203,大洋洲 OCEANIA,帛琉
1457,2017,Palau,9884,大洋洲 OCEANIA,帛琉
1458,2018,Palau,11524,大洋洲 OCEANIA,帛琉
1459,2019,Palau,15511,大洋洲 OCEANIA,帛琉


In [63]:
for col in list(df_out_melt):
    print(df_out_melt[col].value_counts())

2009    23
1999    23
2004    23
1993    23
2017    23
2008    23
1995    23
2000    23
2005    23
1980    23
1981    23
2002    23
2019    23
2015    23
1997    23
2010    23
1988    23
2007    23
1985    23
2020    23
1982    23
2011    23
2016    23
1996    23
2006    23
1990    23
1987    23
1984    23
2001    23
1983    23
2018    23
1992    23
1986    23
1991    23
1994    23
2013    23
1998    23
2003    23
2014    23
1989    23
2012    23
Name: Year, dtype: int64
 Malaysia          41
 Netherlands       41
 Switzerland       41
 Korea             41
 Vietnam           41
 Italy             41
New Zealand        41
 Singapore         41
 U.K.              41
 Philippines       41
 Brunei            41
Hong Kong          41
 Canada            41
 Macao             41
 Palau             41
 Germany           41
 Mainland China    41
Japan              41
 U.S.A.            41
 Thailand          41
 Austria           41
 Indonesia         41
 Australia         41
Name: Destination,

In [64]:
df_in_melt.to_excel('Inbound_clean.xlsx')
df_out_melt.to_excel('Outbound_clean.xlsx')